In [20]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

fatal: destination path 'Qualcomm-DL-Hackathon' already exists and is not an empty directory.


In [21]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [22]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [23]:
sample_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/sample_submission.csv')
sample_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [24]:
import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [25]:
import shutil
import os

# Source and destination directories
source_dir1 = '/content/Qualcomm-DL-Hackathon/train/images/images part-1'
source_dir2 = '/content/Qualcomm-DL-Hackathon/train/images/images part-2'
destination_dir = '/content/Qualcomm-DL-Hackathon/train/images'


def copy_files(source, destination):
    for filename in os.listdir(source):
        source_file = os.path.join(source, filename)
        destination_file = os.path.join(destination, filename)
        if os.path.isfile(source_file):
          shutil.copy(source_file, destination_dir)


# Copy files from 'images part-1'
copy_files(source_dir1, destination_dir)

# Copy files from 'images part-2'
copy_files(source_dir2, destination_dir)

print("Files copied successfully!")

Files copied successfully!


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os

# Assuming your image folders are named '0' and '1'
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        # Check if there's a second column for labels
        if self.data.shape[1] > 1:
            label = self.data.iloc[idx, 1]
        else:
            label = 0  # or None, depending on how you want to handle it

        if self.transform:
            image = self.transform(image)
        return image, label

# Data transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Create datasets and dataloaders
train_dataset = ImageDataset(csv_file='/content/Qualcomm-DL-Hackathon/train/train.csv',
                             root_dir='/content/Qualcomm-DL-Hackathon/train/images',
                             transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [13]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128) # Adjust input size based on image dimensions and pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # 2 output classes

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16) # Adjust input size based on image dimensions and pooling
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
import torchvision.models as models

# Initialize the ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True) # Use a pretrained ResNet18

# Modify the fully connected layer for your specific task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # 2 output classes
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]


In [15]:
# VGG model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(pretrained=True)

# Modify the classifier for your specific task
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2) # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 91.8MB/s]


In [27]:
# AlexNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.alexnet(pretrained=True)

# Modify the classifier for your specific task
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2) # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
# LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        # Calculate the correct input size for fc1 dynamically
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        # Reshape to the correct size
        x = x.view(-1, 16 * 13 * 13)

        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the LeNet-5 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [45]:
import torchvision.models as models

# MobileNetV2 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier for your specific task
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 131MB/s]


In [49]:
# GoogleNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.googlenet(pretrained=True)

# Modify the fully connected layer for your specific task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 167MB/s]


In [62]:
# DenseNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained=True) # Use a pretrained DenseNet121

# Modify the classifier for your specific task
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2) # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 167MB/s]


In [63]:
# Training loop (not for lenet)
num_epochs = 100 # Adjust as needed
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    if loss.item() < 0.0001:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.5f}")
            print("Loss is less than 0.001. Breaking the loop.")
            break

    if loss.item() < 0.0001:
        break
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.5f}")

print("Training finished.")

Epoch [1/100], Loss: 0.51041
Epoch [2/100], Loss: 0.16681
Epoch [3/100], Loss: 0.05228
Epoch [4/100], Loss: 0.14610
Epoch [5/100], Loss: 0.29178
Epoch [6/100], Loss: 0.13473
Epoch [7/100], Loss: 0.02112
Epoch [8/100], Loss: 0.06910
Epoch [9/100], Loss: 0.27098
Epoch [10/100], Loss: 0.03211
Epoch [11/100], Loss: 0.01100
Epoch [12/100], Loss: 0.18704
Epoch [13/100], Loss: 0.14087
Epoch [14/100], Loss: 0.05665
Epoch [15/100], Loss: 0.00016
Epoch [16/100], Loss: 0.00011
Epoch [17/100], Loss: 0.00115
Epoch [18/100], Loss: 0.00014
Epoch [19/100], Loss: 0.00004
Loss is less than 0.001. Breaking the loop.
Training finished.


In [64]:
test_dataset = ImageDataset(csv_file='/content/Qualcomm-DL-Hackathon/test.csv',
                            root_dir='/content/Qualcomm-DL-Hackathon/train/images',
                            transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(test_dataset)

# Make predictions on the test data
predictions = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a submission dataframe
submission_df = pd.DataFrame({'image_names': test_data['image_names'], 'emergency_or_not': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
